# MedSAM Training & Testing

In [ ]:
import os
os.chdir("/home/ra-ugrad/Documents/Haleigh/MedicalImage")

# MedSAM Base Testing
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from segment_anything import sam_model_registry
from torch.nn import Linear
from torch.nn import Embedding
from segment_anything.modeling.mask_decoder import MLP

import src.dataloader_local as dl_local
from src.dataloader_local import CancerDataset
from src.medsam_helper import medsam_test_results

In [2]:
mri_data = dl_local.get_mri_data()
mri_data = dl_local.clean_mri_data(mri_data)

train_data, test_data = dl_local.train_test(mri_data)
batch_size = 8

train_dataset = CancerDataset(labels=train_data, path="/home/ra-ugrad/Documents/Segmentations/")
test_dataset = CancerDataset(labels=test_data, path="/home/ra-ugrad/Documents/Segmentations/")
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [3]:
MedSAM_CKPT_PATH = "/home/ra-ugrad/Documents/Haleigh/MedicalImage/models/medsam_vit_b.pth"
medsam_model = sam_model_registry['vit_b'](checkpoint=MedSAM_CKPT_PATH)

medsam_model.mask_decoder.num_mask_tokens = 8
medsam_model.mask_decoder.num_multimask_outputs = 7

medsam_model.image_encoder.patch_embed.proj = nn.Conv2d(3, 768, kernel_size = (35, 35), stride = (3, 3))
medsam_model.mask_decoder.mask_tokens = Embedding(medsam_model.mask_decoder.num_mask_tokens, 256)
medsam_model.mask_decoder.output_hypernetworks_mlps = nn.ModuleList([MLP(256, 256, 32, 3) for i in range(medsam_model.mask_decoder.num_mask_tokens)])
medsam_model.mask_decoder.iou_prediction_head.layers[2] = Linear(in_features=256, out_features=medsam_model.mask_decoder.num_mask_tokens, bias=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
medsam_model = medsam_model.to(device)

In [ ]:
# add model training here


In [ ]:
medsam_model.eval()
all_base_results = medsam_test_results(test_loader, medsam_model, device)
grouped_base_results = all_base_results.groupby(["Patient", "Brightness"]).mean()

/home/ra-ugrad/Documents/Haleigh/MedicalImage/src/medsam_helper.py:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  acc = (torch.tensor(medsam_seg) == y).float().mean(dim=(1, 2))
/home/ra-ugrad/Documents/Haleigh/MedicalImage/src/medsam_helper.py:68: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, pd.DataFrame({"Patient": patient, "Brightness": b_level, "Accuracy": acc})])
/home/ra-ugrad/Documents/Haleigh/MedicalImage/src/medsam_helper.py:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone(

In [12]:
grouped = base_results.groupby(["Patient", "Brightness"]).mean()

In [13]:
grouped['Model'] = "Base"

In [14]:
grouped

Accuracy Model
Patient           Brightness                            
ACRIN 6698_277831 NEW Brightness level 6  0.044286  Base
                  NEW Brightness level 7  0.044083  Base